# DETR implementation from scratch 


DETR (DEtection TRansformer)은 기존의 복잡한 객체 탐지 파이프라인을 단순화하기 위해 제안된 모델

#### 핵심 아이디어: 직접적인 집합 예측 (Direct Set Prediction)
기존 모델들은 수많은 후보 영역(앵커 박스 등)을 만들고 후처리(NMS 등)를 통해 최종 결과를 얻는 **간접적인 방식**을 사용  
반면, DETR은 객체 탐지를 "이미지에서 객체 집합을 직접 예측하는 문제"로 재정의하여 이 과정을 단순화함

#### 주요 구성 요소
1.  **CNN 백본 (Backbone)**: 이미지에서 특징(feature)을 추출
2.  **트랜스포머 (Transformer) 인코더-디코더**:
    *   **인코더**는 이미지 전체의 맥락과 객체들 간의 관계를 학습
    *   **디코더**는 '객체 쿼리(object queries)'를 입력받아, 인코더가 처리한 정보를 바탕으로 각 쿼리에 해당하는 객체의 위치와 클래스를 병렬적으로 예측
3.  **이분 매칭 손실 (Bipartite Matching Loss)**:
    *   모델이 예측한 결과와 실제 정답 간에 일대일 매칭을 강제. 이를 통해 중복된 예측을 자연스럽게 방지하므로, NMS 같은 후처리 단계 없음

#### 장점
*   **End-to-End 단순성**: 앵커 박스 생성, NMS 등 수작업으로 설계해야 했던 복잡한 구성 요소들을 제거하여 파이프라인 간소화
*   **높은 성능**: 복잡성을 줄였음에도 불구하고, Faster R-CNN과 같은 기존 모델들과 COCO 데이터셋에서 대등한 성능 보임


#### Architecture 

![DETR Architecture](images/detr_architecture.png)

In [31]:
import autoroot
from src.config import dataset_config
from src.config import model_config
from src.config import train_config
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

---

### 1. Backbone

- CNN 기반의 백본
- 입력 이미지(B, 3, H, W)를 받아 feature map으로 변환
- 논문에서는 ResNet-50이나 ResNet-101과 같은 표준 CNN 모델을 사용함.

- 주요 단계
1.  ResNet에서 feature map 추출
2.  1x1 Conv로 feature map의 채널 수를 `model_dim`으로 축소 (결과: B, model_dim, H/32, W/32).
3.  처리된 feature map 을 flatten하여 인코더 입력 시퀀스 형태로 변환, (결과: B, H/32 * W/32, model_dim)


In [32]:
import torch
import torchvision
import torchvision.models as models
import torch.nn as nn
# Backbone of DETR
class Backbone(nn.Module):
    def __init__(self,
                 backbone_dim=model_config['backbone_dim'],
                 backbone_model=model_config['backbone_model'],
                 model_dim=model_config['model_dim']):
        super().__init__()
        # Load the pre-trained resnet50 model without final FC layer and tracking gradients
        if backbone_model == 50:
            resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT, norm_layer=torchvision.ops.FrozenBatchNorm2d)
            # Remove the final fully connected layer
        elif backbone_model == 101:
            resnet = models.resnet101(weights=models.ResNet101_Weights.DEFAULT, norm_layer=torchvision.ops.FrozenBatchNorm2d)
            # Remove the final fully connected layer

        self.resnet = nn.Sequential(*list(resnet.children())[:-2])
        if model_config['freeze_backbone']:
            for param in self.resnet.parameters():
                param.requires_grad = False


        self.embed_proj = nn.Conv2d(in_channels=backbone_dim,
                                   out_channels=model_dim,
                                   kernel_size=1)


    def forward(self, x): 
        print(f"Backbone input: {x.shape}")
        # x is the initial batch input (B, 3, H ,W)
        x = self.resnet(x) # (B, 2048, H', W')
        x = self.embed_proj(x) # (B, embed_dim, H', W')
        x = x.permute(0, 2, 3, 1) # (B, H', W', embed_dim)
        x = x.flatten(1, 2)  # (B, H'*W', embed_dim)
        print(f"Backbone output: {x.shape}")
        return x

---

### 2. Positional Encoding

Transformer는 본질적으로 입력 시퀀스의 순서를 고려하지 못하는 순열 불변(permutation-invariant) 모델  
따라서 이미지 특징의 공간적 위치 정보를 제공하기 위해 Positional Encoding을 추가


In [33]:
class PositionalEncoding(nn.Module):
    def __init__(self, max_len, model_dim=model_config['model_dim']):
        super().__init__()
        self.pe = nn.Parameter(torch.randn(max_len, model_dim))  # (max_len, D)

    def forward(self, x):
        # x: (B, N, D)
        B, N, D = x.size()
        return self.pe[:N].unsqueeze(0).repeat(B, 1, 1)  # (B, N, D)

---

### 3. Transformer Encoder

Transformer 인코더는 백본에서 추출된 이미지 특징 시퀀스를 입력받아, 이미지의 전반적인 맥락과 객체 간의 관계를 학습하는 역할 수행

![Encoder](images/encoder.png)

In [34]:
class EncoderLayer(nn.Module):
    def __init__(self,
                 model_dim=model_config['model_dim'],
                 encoder_heads=model_config['encoder_heads'],
                 mlp_inner_dim=model_config['mlp_inner_dim']):
        super().__init__()
        # Attention layer
        self.self_attn = nn.MultiheadAttention(model_dim, encoder_heads)
        self.norm_attn = nn.LayerNorm(model_dim)

        # MLP layer
        self.linear1 = nn.Linear(model_dim, mlp_inner_dim)
        self.dropout = nn.Dropout(0.1)
        self.linear2 = nn.Linear(mlp_inner_dim, model_dim)
        self.norm_mlp = nn.LayerNorm(model_dim)

    def forward(self, features, pos_enc):
        # features, pos_enc: (B, N, D)
        
        # Attention
        q = (features + pos_enc).transpose(0, 1)  # Add positional encoding only to q and k
        k = (features + pos_enc).transpose(0, 1)  
        v = features.transpose(0, 1)
        attn_output, _ = self.self_attn(q, k, v) # (N, B, D)
        attn_output = features +  attn_output.transpose(0, 1) # (B, N ,D)
        attn_output = self.norm_attn(attn_output)

        # MLP
        ff = self.linear2(self.dropout(torch.relu(self.linear1(attn_output)))) # FC1 -> ReLU -> Dropout -> FC2 
        encoder_output = self.norm_mlp(attn_output + ff)
        return encoder_output

In [35]:
class Encoder(nn.Module):
    def __init__(self,
                 model_dim=model_config['model_dim'],
                 encoder_heads=model_config['encoder_heads'],
                 encoder_layers=model_config['encoder_layers']):
        super().__init__()
        self.layers = nn.ModuleList([EncoderLayer(model_dim, encoder_heads) for _ in range(encoder_layers)])

    def forward(self, features, pos_enc):
        print(f"Encoder input: {features.shape}")
        for layer in self.layers:
            features = layer(features, pos_enc) # The positional encoder is added before every attention layer.
        print(f"Encoder output: {features.shape}")
        return features

---

### 4. Transformer Decoder

Transformer 디코더는 인코더의 출력과 '객체 쿼리(object queries)'를 입력받아,  
최종적으로 각 객체의 클래스와 바운딩 박스를 예측하는 역할

**Multi-Head Self-Attention**:  
- 객체 쿼리들 사이의 상호 관계를 학습함. 이 과정을 통해 모델은 여러 쿼리가 서로 다른 객체에 집중하도록 유도하여 중복된 예측을 방지하는 능력을 기름.
- Query, Key, Value가 모두 이전 디코더 레이어의 출력(또는 초기 입력 `tgt`)으로부터 생성됨.



![Decoder](images/decoder.png)

In [36]:
class DecoderLayer(nn.Module):
    def __init__(self,
                 model_dim=model_config['model_dim'],
                 decoder_heads=model_config['decoder_heads'],
                 mlp_inner_dim=model_config['mlp_inner_dim']):
        super().__init__()

        # Self Attention
        self.self_attn = nn.MultiheadAttention(model_dim, decoder_heads)
        self.norm_self_attn = nn.LayerNorm(model_dim)
        
        # Cross Attention
        self.cross_attn = nn.MultiheadAttention(model_dim, decoder_heads)
        self.norm_cross_attn = nn.LayerNorm(model_dim)
        
        # MLP
        self.linear1 = nn.Linear(model_dim, mlp_inner_dim)
        self.dropout = nn.Dropout(0.1)
        self.linear2 = nn.Linear(mlp_inner_dim, model_dim)
        self.norm_mlp = nn.LayerNorm(model_dim)

    def forward(self, tgt, memory, pos_enc, query_pos):
        #print(f"Initial tgt value (first query, first 5 values): {tgt[0, 0, :5].detach().cpu().numpy()}")

        # Self-attention
        q = (tgt + query_pos).transpose(0, 1) # query positional encoding is only added to q and k
        k = (tgt + query_pos).transpose(0, 1) # (N, B, D)
        v = tgt.transpose(0, 1) 
        self_attn_output, _ = self.self_attn(q, k, v) # (N, B, D)
        tgt = self.norm_self_attn(tgt + self_attn_output.transpose(0, 1)) # (B, N ,D)
        #print(f"tgt after MHSA value (first query, first 5 values): {tgt[0, 0, :5].detach().cpu().numpy()}")

        # Cross-attention
        q = (tgt + query_pos).transpose(0, 1)         # the query positional encoding is added to object queries
        k = (memory + pos_enc).transpose(0, 1) # the positional encoding is added to the encoder's output
        v = memory.transpose(0, 1) # (N, B ,D)
        cross_attn_output, cross_attn = self.cross_attn(q, k, v) # (N, B ,D)
        tgt = self.norm_cross_attn(tgt + cross_attn_output.transpose(0, 1)) # (B, N ,D)

        # MLP
        ff = self.linear2(self.dropout(torch.relu(self.linear1(tgt))))
        tgt = self.norm_mlp(tgt + ff) # (B, N, D)
        return tgt, cross_attn

In [37]:
class Decoder(nn.Module):
    def __init__(self,
                 model_dim=model_config['model_dim'],
                 decoder_heads=model_config['decoder_heads'],
                 decoder_layers=model_config['decoder_layers']):
        super().__init__()
        self.layers = nn.ModuleList([DecoderLayer(model_dim, decoder_heads) for _ in range(decoder_layers)])

        self.output_norm = nn.LayerNorm(model_dim)
        
    def forward(self, tgt, memory, pos_enc, query_pos):
        print(f"Decoder input: tgt={tgt.shape}, memory={memory.shape}")
        outputs = []
        cross_attn_weights = []
        for layer in self.layers:
            tgt, decoder_cross_attn = layer(tgt, memory, pos_enc, query_pos)
            cross_attn_weights.append(decoder_cross_attn)
            outputs.append(self.output_norm(tgt))

        output = torch.stack(outputs)
        print(f"Decoder output: {output.shape}")
        return output, torch.stack(cross_attn_weights)

---

### 5. DETR Class 


In [38]:
class DETR(nn.Module):
    def __init__(self,
                 model_dim=model_config['model_dim'],
                 num_queries=model_config['num_queries'],
                 num_classes=dataset_config['num_classes'],
                 max_len=5000):
        super().__init__()

        self.backbone = Backbone()

        self.encoder = Encoder()
        self.decoder = Decoder()

        # Learnable positional encoding for feature embeddings and object queries
        self.pos_enc = PositionalEncoding(max_len, model_dim)  # Learnable positional encoding

        self.query_embed = nn.Embedding(num_queries, model_dim)  # Learnable queries

        # Prediction
        self.class_mlp = nn.Linear(model_dim, num_classes)
        self.bbox_mlp = nn.Sequential(nn.Linear(model_dim, model_dim),
                                        nn.ReLU(),
                                        nn.Linear(model_dim, model_dim),
                                        nn.ReLU(),
                                        nn.Linear(model_dim, 4))

    def forward(self, x):
        # x: (B, 3, H, W)
        B = x.size(0)

        # Backbone
        features = self.backbone(x)                 # (B, N, D)

        pos_enc = self.pos_enc(features)            # (B, N, D) ← now learnable

        # Encoder
        memory = self.encoder(features, pos_enc)    # (B, N, D)

        query_pos = self.query_embed.weight.unsqueeze(0).repeat(B, 1, 1)  # (B, num_queries, D)
        tgt = torch.zeros_like(query_pos)

        # Decoder
        hs, _ = self.decoder(tgt, memory, pos_enc, query_pos)  # (num_decoders, B, num_queries, D)

        # Prediction
        pred_logits = self.class_mlp(hs)  # (num_decoders, B, num_queries, num_classes)
        pred_bboxes = self.bbox_mlp(hs).sigmoid()  # (num_decoders, B, num_queries, 4)

        return {'pred_logits': pred_logits, 'pred_boxes': pred_bboxes}

---

### 행렬 값 계산

In [39]:
import torch

test = torch.randn((16, 3, 640, 640))

model_detr = DETR().to(device)
test_output = model_detr(test.to(device))
print(test_output['pred_logits'].shape)  # Should be (num_decoders, B, num_queries, num_classes)
print(test_output['pred_boxes'].shape)  # Should be (num_decoders, B, num_queries, 4)

Backbone input: torch.Size([16, 3, 640, 640])
Backbone output: torch.Size([16, 400, 256])
Encoder input: torch.Size([16, 400, 256])
Encoder output: torch.Size([16, 400, 256])
Decoder input: tgt=torch.Size([16, 25, 256]), memory=torch.Size([16, 400, 256])
Decoder output: torch.Size([4, 16, 25, 256])
torch.Size([4, 16, 25, 21])
torch.Size([4, 16, 25, 4])


In [40]:
from torchinfo import summary

summary(model=model_detr, 
        input_size=(train_config['batch_size'], 3, dataset_config['im_size'], dataset_config['im_size']),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
) 

Backbone input: torch.Size([60, 3, 640, 640])
Backbone output: torch.Size([60, 400, 256])
Encoder input: torch.Size([60, 400, 256])
Encoder output: torch.Size([60, 400, 256])
Decoder input: tgt=torch.Size([60, 25, 256]), memory=torch.Size([60, 400, 256])
Decoder output: torch.Size([4, 60, 25, 256])


Layer (type (var_name))                                 Input Shape          Output Shape         Param #              Trainable
DETR (DETR)                                             [60, 3, 640, 640]    [4, 60, 25, 4]       6,400                Partial
├─Backbone (backbone)                                   [60, 3, 640, 640]    [60, 400, 256]       --                   Partial
│    └─Sequential (resnet)                              [60, 3, 640, 640]    [60, 2048, 20, 20]   --                   False
│    │    └─Conv2d (0)                                  [60, 3, 640, 640]    [60, 64, 320, 320]   (9,408)              False
│    │    └─FrozenBatchNorm2d (1)                       [60, 64, 320, 320]   [60, 64, 320, 320]   --                   --
│    │    └─ReLU (2)                                    [60, 64, 320, 320]   [60, 64, 320, 320]   --                   --
│    │    └─MaxPool2d (3)                               [60, 64, 320, 320]   [60, 64, 160, 160]   --                   --
│

---

### Matching cost and Loss

- 매칭 (Hungarian 알고리즘)
> - 예측 쿼리와 정답 박스 사이의 비용을 계산
> - 분류 비용: 예측 클래스와 실제 클래스 차이 (Cross Entropy)
> - L1 비용: 예측 박스 좌표와 실제 박스 좌표 차이
> - GIoU 비용: 두 박스의 겹침 정도 (1 - GIoU)
> - 세 비용에 가중치를 곱해 합산한 뒤 ( 비용 행렬 ) , Hungarian 알고리즘으로 최적 매칭 구함
> - Hungarian 알고리즘은 이 행렬에서 전체 합이 최소가 되도록 예측과 정답을 1:1 대응시킴  
> - 결과적으로 일부 쿼리는 GT와 매칭되고, 나머지는 배경으로 처리됨 


- 손실 계산
> - 매칭된 쌍에 대해서는
> > - 분류 손실: CE loss
> > - 박스 좌표 손실: L1 loss
> > - 박스 겹침 손실: GIoU loss  
를 계산하고 평균낸다.
> - 매칭되지 않은 쿼리는 배경 클래스로 학습

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from scipy.optimize import linear_sum_assignment
from collections import defaultdict

class DETRLoss(nn.Module):
    def __init__(self,
                 num_classes=dataset_config['num_classes'],
                 decoder_layers=model_config['decoder_layers'],
                 num_queries=model_config['num_queries'],
                 cls_cost_weight=model_config['cls_cost_weight'],
                 l1_cost_weight=model_config['l1_cost_weight'],
                 giou_cost_weight=model_config['giou_cost_weight'],
                 bg_class_idx=dataset_config['bg_class_idx'],
                 bg_class_weight=model_config['bg_class_weight'],
                 nms_threshold=model_config['nms_threshold']):
        super().__init__()
        # 클래스 수, 디코더 층 수, 쿼리 수 등 설정값 저장
        self.num_classes = num_classes
        self.num_decoder_layers = decoder_layers
        self.num_queries = num_queries
        self.cls_cost_weight = cls_cost_weight
        self.l1_cost_weight = l1_cost_weight
        self.giou_cost_weight = giou_cost_weight
        self.bg_class_idx = bg_class_idx
        self.bg_class_weight = bg_class_weight
        self.nms_threshold = nms_threshold

    def compute_hungarian_matching(self, pred_logits, pred_boxes, targets):
        """
        예측 쿼리와 GT 박스를 헝가리안 매칭으로 연결
        pred_logits: (B, num_queries, num_classes)
        pred_boxes: (B, num_queries, 4) in cxcywh
        targets: [{'labels': (N_i,), 'boxes': (N_i,4 in xyxy)}, ...]
        """

        batch_size = pred_logits.shape[0]
        num_queries = pred_logits.shape[1]

        # 클래스 확률로 변환: (B*num_queries, num_classes)
        class_prob = pred_logits.reshape(-1, self.num_classes).softmax(dim=-1)
        pred_boxes = pred_boxes.reshape(-1, 4)

        # 배치 내 모든 GT 라벨과 박스를 하나로 모음
        target_labels = torch.cat([t['labels'] for t in targets])
        target_boxes = torch.cat([t['boxes'] for t in targets])

        # (1) 분류 비용: 정답 클래스 확률이 높을수록 비용 ↓
        cost_classification = -class_prob[:, target_labels]

        # (2) 박스 좌표 변환 (cxcywh → xyxy)
        pred_boxes_xyxy = torchvision.ops.box_convert(pred_boxes, 'cxcywh', 'xyxy')

        # (3) L1 비용 (박스 좌표 차이 절댓값)
        cost_l1 = torch.cdist(pred_boxes_xyxy, target_boxes, p=1)

        # (4) GIoU 비용 (겹칠수록 낮음)
        cost_giou = -torchvision.ops.generalized_box_iou(pred_boxes_xyxy, target_boxes)

        # (5) 총 비용 = 분류 + L1 + GIoU (가중합)
        total_cost = (self.cls_cost_weight * cost_classification +
                      self.l1_cost_weight * cost_l1 +
                      self.giou_cost_weight * cost_giou)

        # (B, num_queries, total_num_gts)
        total_cost = total_cost.reshape(batch_size, self.num_queries, -1).cpu()

        # 각 이미지별 GT 개수
        num_targets_per_image = [len(t['labels']) for t in targets]

        # 이미지별 비용 행렬 분리
        total_cost_per_image = total_cost.split(num_targets_per_image, dim=-1)

        match_indices = []
        for b in range(batch_size):
            # linear_sum_assignment → 비용 최소화 매칭 수행
            pred_inds, tgt_inds = linear_sum_assignment(total_cost_per_image[b][b])
            # pred_inds: 선택된 쿼리 인덱스, tgt_inds: 대응되는 GT 인덱스
            match_indices.append((
                torch.as_tensor(pred_inds, dtype=torch.int64),
                torch.as_tensor(tgt_inds, dtype=torch.int64)
            ))
        return match_indices

    def compute_losses(self, pred_logits, pred_boxes, targets, match_indices):
        """
        매칭 결과를 이용해 분류 손실과 박스 손실 계산
        """
        batch_size = pred_logits.shape[0]

        classification_losses = []
        bbox_losses = []
        bbox_giou_losses = []

        for b in range(batch_size):
            pred_idx, tgt_idx = match_indices[b]  # 매칭된 쿼리와 GT 인덱스

            # 기본은 전부 배경 클래스로 채움
            target_classes = torch.full((self.num_queries,), self.bg_class_idx,
                                        dtype=torch.int64, device=pred_logits.device)
            # 매칭된 쿼리만 정답 라벨로 덮어씀
            target_classes[pred_idx] = targets[b]['labels'][tgt_idx]

            # 클래스 가중치 설정 (배경은 작은 값)
            cls_weights = torch.ones(self.num_classes, device=pred_logits.device)
            cls_weights[self.bg_class_idx] = self.bg_class_weight

            # (1) 분류 손실 (쿼리별 cross-entropy, 마지막에 평균)
            loss_cls = F.cross_entropy(pred_logits[b], target_classes,
                                       weight=cls_weights, reduction='none')
            classification_losses.append(loss_cls)

            # 매칭된 박스만 사용
            matched_pred_boxes = pred_boxes[b][pred_idx]
            target_boxes = targets[b]['boxes'][tgt_idx]

            # 좌표 변환 (cxcywh → xyxy)
            pred_xyxy = torchvision.ops.box_convert(matched_pred_boxes, 'cxcywh', 'xyxy')

            # (2) 박스 손실
            loss_bbox = F.l1_loss(pred_xyxy, target_boxes, reduction='none').sum(dim=1)
            loss_giou = torchvision.ops.generalized_box_iou_loss(pred_xyxy, target_boxes, reduction='none')

            bbox_losses.append(loss_bbox)
            bbox_giou_losses.append(loss_giou)

        # 배치 전체 평균
        all_cls_loss = torch.cat(classification_losses).mean()
        all_l1_loss = torch.cat(bbox_losses).mean()
        all_giou_loss = torch.cat(bbox_giou_losses).mean()

        # 가중치 적용
        los_cls = all_cls_loss * self.cls_cost_weight
        los_bbox = all_l1_loss * self.l1_cost_weight + all_giou_loss * self.giou_cost_weight
        
        return los_cls, los_bbox

    def forward(self, pred_classes, pred_bboxes, targets,
                training=True, score_thresh=0.0, use_nms=False):
        """
        훈련 모드: 손실 반환
        평가 모드: 예측 박스/점수/라벨 반환
        """
        losses = defaultdict(list)
        detections = []
        detr_output = {}

        if training:
            # 모든 디코더 층에서 손실 계산
            for decoder_idx in range(self.num_decoder_layers):
                cls_out = pred_classes[decoder_idx]
                box_out = pred_bboxes[decoder_idx]

                with torch.no_grad():
                    match_indices = self.compute_hungarian_matching(cls_out, box_out, targets)

                loss_cls, loss_bbox = self.compute_losses(cls_out, box_out, targets, match_indices)
                losses['classification'].append(loss_cls)
                losses['bbox_regression'].append(loss_bbox)

            detr_output['loss'] = losses

        else:
            # 평가 시엔 마지막 디코더 층만 사용
            cls_out = pred_classes[-1] 
            box_out = pred_bboxes[-1]

            # 클래스 확률
            prob = F.softmax(cls_out, -1)

            # 배경 클래스 제거 (bg가 0이면 1번부터 시작)
            if self.bg_class_idx == 0:
                scores, labels = prob[..., 1:].max(-1)
                labels += 1
            else:
                scores, labels = prob[..., :-1].max(-1)

            # 박스 좌표 변환
            boxes = torchvision.ops.box_convert(box_out, 'cxcywh', 'xyxy')

            # 이미지별 결과 정리
            for b in range(boxes.shape[0]):
                score_b, label_b, box_b = scores[b], labels[b], boxes[b]

                # score threshold 적용
                keep = score_b >= score_thresh
                score_b, label_b, box_b = score_b[keep], label_b[keep], box_b[keep]

                # NMS 적용 (옵션)
                if use_nms:
                    keep_nms = torchvision.ops.batched_nms(box_b, score_b, label_b, self.nms_threshold)
                    score_b, label_b, box_b = score_b[keep_nms], label_b[keep_nms], box_b[keep_nms]

                detections.append({"boxes": box_b, "scores": score_b, "labels": label_b})

            detr_output['detections'] = detections

        return detr_output


### DETR VS RT-DETR vs Deformable DETR 



| 모델 | 핵심 아이디어 | 장점 | 한계/특징 |
|------|--------------|------|-----------|
| **DETR (2020)** | - Object detection을 **Set Prediction 문제**로 정의 <br> - CNN 백본 → Transformer 인코더/디코더 → Object queries 사용 <br> - **Hungarian 매칭**으로 예측-정답 1:1 대응 | - Anchor/Proposal 불필요 (엔드투엔드) <br> - 구조가 단순함 | - 학습 수렴이 매우 느림 (300 epoch 이상) <br> - 작은 객체 탐지 성능 낮음 |
| **Deformable DETR (2020)** | - 전역 어텐션 대신 **소수의 변형 가능한 샘플링 포인트(Deformable Attention)** 만 집중 <br> - **Multi-scale feature map** 활용 → 작은 객체 탐지 강화 | - 학습 속도 10배 이상 향상 (50 epoch 내 수렴) <br> - 작은 객체 성능 개선 | - 구조가 복잡해짐 <br> - 여전히 실시간성은 부족 |
| **RT-DETR (2023)** | - 실시간(real-time) 탐지에 최적화 <br> - Transformer 디코더 대신 **IoU-aware Detection Head** 사용 → 박스 품질(IoU)까지 직접 예측 <br> - 클래스 확률 + 박스 좌표 + IoU를 동시에 출력 | - 50FPS 이상 달성 (COCO 벤치마크) <br> - 정확도와 속도의 균형 우수 | - 극도로 작은 객체, 복잡한 장면에서 성능 한계 존재 |

